# Dados faltantes


**Notebook destinado ao estudo de técnicas para lidar com dados faltantes** 

###  Breve explicação sobre o tema:



Um dos problemas mais comuns que surgem durante as análises de dados, é a presença de valores faltantes. Não ha uma fórmula fechada que nos mostre como tratar a ausência de valores. Existem diversas abordagens e cada uma delas será mais interessante de acordo com os dados e objetivos da análise.

O primeiro passo para entender e pensar em estratégias para lidar com missing data é entender o motivo desta ausência, podemos não ter as informações por diferentes motivos, e as algumas vezes, a ausência do dado é uma informação, portanto, antes de pensar em como tratar estes casos precisamos entender os tipos de dados faltantes com os quais podemos esbarrar:

**Tipos de dados faltantes**


* **MCAR** - Missing Completely Random

Dados perdidos de forma completamente aleatória. Neste caso não há justificativa para a ocorrência, o fato do valor estar ausente não tem nada a ver com seu valor ou com outras variáveis.  Como exemplo para este caso, podemos citar dados perdidos durante integração entre sistemas ou na imputação dos dados.

* **MAR** - Missing at Random

Quando temos dados perdidos de forma aleatória, que dizer que a ausência do valor não tem a ver com o valor em sí, não se sabe se o dado deveria existir ou não. Nestes casos os dados não estão faltantes em todas as observações, geralmente apenas em alguns casos. Sendo assim, pode-se usar os dados observados parar fazer alguma análise preditiva e imputar os valores ausentes.
 Como exemplo deste tipo de dados faltantes, podemos pensar num questionário onde alguém não viu o campo de idade em branco. É possível que com os outros campos preenchidos, possamos estimar a idade deste indivíduo.

* **MNAR** - Missing not at Random

Pode ser também que a ausência da informação em sí, seja uma informação. Nestes casos a falta de informação pode ter algum motivo especifico. O motivo pode ter a ver com o valor em sí ( Por exemplo, uma pessoa com a renda muito alta não querer revelar este valor, ou pessoas LGBTQIA+ não estarem confortáveis em responder sobre sua sexualidade), ou pode ter a ver com alguma outra causa. Suponhamos que mulheres entre 45 e 55  anos não responderam a uma pergunta específica. 
Nestes casos os dados faltantes podem trazer alguma informação mesmo que não seja de forma direta. Se removermos estes casos, podemos viciar o nosso grupo de dados. No exemplo das mulheres entre 44 e 55 anos, teríamos uma base que não seria capaz de representar tal grupo.


Após entendermos um pouco dos tipos de dados faltantes, podemos olhar para cada caso e traçar a estratégia a ser utilizada. Não podemos definir qual a estratégia é ideal para cada caso, mas devemos tentar entender a estrutura desta ausência de dados e traçar a estratégia que parecer melhor se encaixar no problema, as estratégia vão desde eliminar os valores faltantes até técnicas avançadas para prever os valores faltantes.

Vamos explorar a seguir algumas das possíveis maneiras de lidar com estes problemas, mas exitem N outros métodos que podem ser usados.


## Carregando pacotes

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

### Algumas configurações padrão
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.2f' % x)

Versão python 3.8.3